# Query MapRDB Using Spark SQL
This notebook is still a work in progress, and requires some "unsupported" configurations in the mapr cluster before it can run against it. At a high level, i'm using mapr-livy (running at the testuser) on a cluster node to submit spark jobs in the cluster, and load the data from maprdb. I also had to pass in hive-site.xml to obtain the connection to the hive metastore, file which contains plaintext passwords. For now, please use this notebook as a conceptual demo only. 

In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

In [4]:
%load_ext sparkmagic.magics

In [53]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HBox(childre…

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1523343062059_0001,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Verify connection to hive mestatore

In [5]:
%%spark -s testing -c sql -o localvar 
show tables
#you should be able to see existing tables in hive

UsageError: Cell magic `%%spark` not found.


### Load all_transactions_table from maprdb into a spark dataframe, and register as a temporary view

In [48]:
%%spark -s testing
df = spark.loadFromMapRDB("/user/testuser/eth/all_transactions_table")
df.createOrReplaceTempView("all_transactions_view")
df.count()
#df.printSchema()
#sqlDF = spark.sql("SELECT * from all_transactions_view")
#sqlDF.show()

<console>:25: error: not found: value df
val $ires6 = df
             ^
<console>:23: error: not found: value df
       df = spark.loadFromMapRDB("/user/testuser/all_blocks")
       ^


In [7]:
%%spark -s testing -c sql -o localvar
create table all_transactions_hive_table as select * from all_transactions_view

""


In [42]:
%%spark -s testing -c sql -o localvar
select count(*) from all_transactions_sparkie

An error was encountered:
Invalid status code '404' from http://172.16.9.42:8998/sessions/0 with error payload: "Session '0' not found."


In [41]:
%%spark
df = spark.loadFromMapRDB("/user/testuser/all_txs")
df.createOrReplaceTempView("all_txs_view2")
sqlDF = spark.sql("SELECT * from all_txs_view2")
df.write.parquet("/user/testuser/all_parquet_x")

In [14]:
%%spark

NameError: name 'df' is not defined

In [ ]:
%manage_spark

In [ ]:
%%spark
sc.version